In [33]:
import matplotlib.pyplot as plt
import pandas as pd 
import spotipy 
import librosa
import librosa.display
import numpy as np
import matplotlib as mpl
import urllib.request
import seaborn as sns
import tqdm
import scipy
import joblib
from pathlib import Path
from IPython.display import Audio, Markdown, Image
from spotipy.oauth2 import SpotifyClientCredentials 


# Números y Datos
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.interpolate import interp1d

# Análisis de sonido
import librosa
import librosa.display
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials 

from scipy.signal import savgol_filter

# Machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix

sns.set_context('poster')
sns.set_style('darkgrid')

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

In [65]:
df_int = pd.read_pickle('../sources/dataset_integrado.pickle')
genre_real = df_int['genre']
df_int = df_int.drop([
    'disc_number',
    'duration_ms',
    'popularity',
    'track_number',
],1)

In [66]:
from pyclustering.cluster.clarans import clarans;
from pyclustering.utils import timedcall;
from sklearn import datasets


In [89]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

newdf = df_int.select_dtypes(include=numerics)
newdf.head(3).T.head(20)

# newdf = newdf.iloc[:,0:12]

id,4d3XHYFFuYYzxWr2cJ6yQl,3X3p3u03P8eFL8WTH0oaaU,4KlXeaheot0OI9PoOWspvZ
danceability,0.48700,0.615000,0.79200
energy,0.14500,0.204000,0.30100
key,5.00000,7.000000,9.00000
loudness,-18.67000,-16.075000,-9.92500
mode,1.00000,1.000000,0.00000
speechiness,0.05220,0.043900,0.04090
acousticness,0.87600,0.872000,0.54600
instrumentalness,0.82200,0.382000,0.82100
liveness,0.14900,0.071000,0.07700
valence,0.14200,0.496000,0.42400


In [92]:
from sklearn.preprocessing import quantile_transform

data = pd.DataFrame(
    quantile_transform(
        newdf, 
        output_distribution = 'normal', 
        random_state = 0, 
        copy = True
    ),
    columns=newdf.columns
)
newdf.head(3).T.head(15)

id,4d3XHYFFuYYzxWr2cJ6yQl,3X3p3u03P8eFL8WTH0oaaU,4KlXeaheot0OI9PoOWspvZ
danceability,0.48700,0.615000,0.79200
energy,0.14500,0.204000,0.30100
key,5.00000,7.000000,9.00000
loudness,-18.67000,-16.075000,-9.92500
mode,1.00000,1.000000,0.00000
speechiness,0.05220,0.043900,0.04090
acousticness,0.87600,0.872000,0.54600
instrumentalness,0.82200,0.382000,0.82100
liveness,0.14900,0.071000,0.07700
valence,0.14200,0.496000,0.42400


In [91]:
n=9
np.random.seed(359)

genres = genre_real.unique()

print('Original')
print(confusion_matrix(genre_real.values, genre_real.values))

selected_models = [
    (
        KMedoids(metric="manhattan", n_clusters=n),
        "KMedoids (manhattan)",
    ),
    (
        KMedoids(metric="euclidean", n_clusters=n),
        "KMedoids (euclidean)",
    ),
    (KMedoids(metric="cosine", n_clusters=n), "KMedoids (cosine)"),
    (KMeans(n_clusters=n), "KMeans"),
]

for i, (model, description) in enumerate(selected_models):
    model.fit(data)
    Z = model.predict(data)
    Z2 = [genres[x] for x in Z]
    print(description)
    print(confusion_matrix(genre_real.values, Z2))


Original
[[442   0   0   0   0   0   0   0   0]
 [  0 396   0   0   0   0   0   0   0]
 [  0   0 486   0   0   0   0   0   0]
 [  0   0   0 477   0   0   0   0   0]
 [  0   0   0   0 421   0   0   0   0]
 [  0   0   0   0   0 461   0   0   0]
 [  0   0   0   0   0   0 447   0   0]
 [  0   0   0   0   0   0   0 477   0]
 [  0   0   0   0   0   0   0   0 475]]
KMedoids (manhattan)
[[111  29  27   2  20   6  16   6 225]
 [  7  10   3   5  31   0   1  20 319]
 [  3   2   5   3   3 468   0   0   2]
 [ 64   5  27  28   2 335  14   1   1]
 [ 50  16  34  84  41   8  34  70  84]
 [  3   5   1  11 144   0   2  14 281]
 [183  27  53  62  27  19  17  14  45]
 [ 12   0  51  56  19 310   4  25   0]
 [118  11  60  11   5 254  11   2   3]]
KMedoids (euclidean)
[[139  11  10  12   8  15 194  39  14]
 [  7   2   7   6   2  33 314  10  15]
 [  3 246   0   3 228   0   1   4   1]
 [ 59  47  23  23 294   2   0   6  23]
 [ 50   9  52  69   9  93  69  17  53]
 [  9  12  12  29   1  61 300   9  28]
 [168  17  